In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
import numpy as np 
from numba import jit

@jit
def fast_auc(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    nfalse = 0
    auc = 0
    n = len(y_true)
    for i in range(n):
        y_i = y_true[i]
        nfalse += (1 - y_i)
        auc += y_i * nfalse
    auc /= (nfalse * (n - nfalse))
    return auc

In [3]:
df_id = pd.read_csv('./data/train_identity.csv')
df_trans = pd.read_csv('./data/train_transaction.csv')

In [4]:
test_id = pd.read_csv('./data/test_identity.csv')
test_trans = pd.read_csv('./data/test_transaction.csv')

In [5]:
df_raw = df_trans.join(df_id.set_index('TransactionID'), on='TransactionID', how='left')

In [7]:
df_test_raw = test_trans.join(test_id.set_index('TransactionID'), on='TransactionID', how='left')
ansTransID = df_test_raw['TransactionID']

In [8]:
## na value
nasum = df_raw.isna().sum()
nasum

TransactionID          0
isFraud                0
TransactionDT          0
TransactionAmt         0
ProductCD              0
card1                  0
card2               8933
card3               1565
card4               1577
card5               4259
card6               1571
addr1              65706
addr2              65706
dist1             352271
dist2             552913
P_emaildomain      94456
R_emaildomain     453249
C1                     0
C2                     0
C3                     0
C4                     0
C5                     0
C6                     0
C7                     0
C8                     0
C9                     0
C10                    0
C11                    0
C12                    0
C13                    0
C14                    0
D1                  1269
D2                280797
D3                262878
D4                168922
D5                309841
D6                517353
D7                551623
D8                515614
D9                515614


In [20]:
## na value
df_test_raw.isna().sum()

TransactionID          0
TransactionDT          0
TransactionAmt         0
ProductCD              0
card1                  0
card2               8654
card3               3002
card4               3086
card5               4547
card6               3007
addr1              65609
addr2              65609
dist1             291217
dist2             470255
P_emaildomain      69192
R_emaildomain     370821
C1                     3
C2                     3
C3                     3
C4                     3
C5                     3
C6                     3
C7                     3
C8                     3
C9                     3
C10                    3
C11                    3
C12                    3
C13                 4748
C14                    3
D1                  6031
D2                234769
D3                203142
D4                 76851
D5                224375
D6                381908
D7                446558
D8                432353
D9                432353
D10                12545


In [9]:
toKeepColname = ['P_emaildomain', 'R_emaildomain', 'addr1', 'addr2', 'DeviceType', 'DeviceInfo']
toDropColname = list(set(list(nasum[nasum>100000].index)) - set(toKeepColname))
# toKeepColname = ['TransactionID', 'DeviceType', 'TransactionDT', 'TransactionAmt', 'ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'addr1', 'addr2']
# toDropColname = list(set(df.columns) - set(['isFraud'] + toKeepColname))

In [10]:
df = df_raw.drop(columns=toDropColname)
df = df.drop(columns='TransactionID')
df_test = df_test_raw.drop(columns=toDropColname)
df_test = df_test.drop(columns='TransactionID')

In [14]:
print("# cols: ", len(df.columns))

# cols:  184


In [15]:
df

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D10,D15,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,DeviceType,DeviceInfo
0,0,86400,68.50,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,13.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,117.000000,0.000000,0.0,0.000000,0.0,0.0,117.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,117.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,117.0,0.0,0.000000,0.000000,0.000000,NaN,NaN
1,0,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,NaN,NaN
2,0,86469,59.00,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,outlook.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,315.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,NaN,NaN
3,0,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,yahoo.com,NaN,2.0,5.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,112.0,84.0,111.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,48.0,28.0,0.0,10.0,4.0,1.0,38.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,50.000000,1758.0000

In [17]:
len(df['DeviceInfo'].unique())

1787

In [19]:
df['DeviceType'].unique()

array([nan, 'mobile', 'desktop'], dtype=object)

In [ ]:
# cat_col = ['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'id_12', 'id_15', 'id_28', 'id_29', 'id_31', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo']
cat_col = ['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'DeviceType']

In [ ]:
for cat in cat_col:
    print(cat)
    df = pd.concat([df, pd.get_dummies(df[cat], prefix=cat,dummy_na=True)],axis=1).drop([cat],axis=1)

In [ ]:
for cat in cat_col:
    print(cat)
    df_test = pd.concat([df_test, pd.get_dummies(df_test[cat], prefix=cat,dummy_na=True)],axis=1).drop([cat],axis=1)

In [ ]:
import datetime
START_DATE = '2018-02-16'
startdate = datetime.datetime.strptime(START_DATE, "%Y-%m-%d")
df['Date'] =df['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds=x)))
df['day_of_week'] = df['Date'].dt.dayofweek
df['hour_a_day'] = df['Date'].dt.hour
df['day_of_month'] = df['Date'].dt.day
df = df.drop(columns='Date')
df = df.drop(columns='TransactionDT')
df = df.fillna(df.median())

In [ ]:
df.median()

In [ ]:
import datetime
START_DATE = '2018-02-16'
startdate = datetime.datetime.strptime(START_DATE, "%Y-%m-%d")
df_test['Date'] = df_test['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds=x)))
df_test['day_of_week'] = df_test['Date'].dt.dayofweek
df_test['hour_a_day'] = df_test['Date'].dt.hour
df_test['day_of_month'] = df_test['Date'].dt.day
df_test = df_test.drop(columns='Date')
df_test = df_test.drop(columns='TransactionDT')
df_test = df_test.fillna(df_test.median())

In [ ]:
# df=(df-df.mean())/df.std()
# df_test=(df_test-df_test.mean())/df_test.std()

In [ ]:
import numpy as np

In [ ]:
labels = np.array(df['isFraud'])
df= df.drop('isFraud', axis = 1)
feature_list = list(df.columns)
features = np.array(df)
real_test_features = np.array(df_test)

In [ ]:
from sklearn.model_selection import train_test_split
## Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.1, random_state = 3)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestRegressor(n_estimators = 10, random_state = 42)
rf = RandomForestClassifier(n_estimators=10, n_jobs=4, random_state=4)
# Train the model on training data
rf.fit(train_features, train_labels);
predictions_1 = rf.predict(test_features)
predictions_2 = 1-rf.predict_proba(test_features)[:,0]


In [ ]:
print('accuracy = ', sum(predictions_1 == test_labels) / len(predictions_1))

In [ ]:
fast_auc(test_labels, predictions_1)

In [ ]:
# predictions = rf.predict(test_features)
rf.fit(features, labels);
predictions = rf.predict(real_test_features)

In [ ]:
# predictions

In [ ]:
# print('accuracy = ', sum(predictions == test_labels) / len(predictions))

In [ ]:
import collections

In [ ]:
collections.Counter(labels)

In [ ]:
collections.Counter(predictions)

In [ ]:
20663/506204

In [ ]:
ans = rf.predict(real_test_features)

## Prob
# ans = 1-rf.predict_proba(real_test_features)[:,0]

# ans

In [ ]:
z=pd.DataFrame({'TransactionID': ansTransID, 'isFraud': ans})

In [ ]:
z.to_csv('./submission0926.csv')

In [ ]:
len(z[z['isFraud']>0.5])

In [ ]:
z

In [ ]:
df.describe()

In [ ]:
collections.Counter(ans)